In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Importando dados

Aqui usaremos o dataset CIFAR10 (https://www.tensorflow.org/datasets/catalog/cifar10).

In [ ]:
# carregando dataset
(trainX, trainY), (testX, testY) = cifar10.load_data()

In [ ]:
# Convertendo valores dos pixels em float
trainX = trainX.astype('float32')
testX = testX.astype('float32')

# normalização para escala [0-1]
trainX = trainX / 255.0
testX = testX / 255.0

# remove a segunda dimensão
trainY = trainY.squeeze()
testY = testY.squeeze()

# transformando a variável alvo (target) para uma codificação one hot
trainY = to_categorical(trainY, num_classes=10)
testY = to_categorical(testY, num_classes=10)

In [ ]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(trainX)

# Construindo o modelo usando Transfer Learning

In [ ]:
pre_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

model = Sequential()

# Mantem apenas as ultimas 4 camadas do VGG16 ativas para o treinamento
for index, layer in enumerate(pre_model.layers):
    if index < len(pre_model.layers) - 4:
        layer.trainable = False
    model.add(layer)

model.add(Flatten())
model.add(Dense(512, activation='relu', name='hidden1'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu', name='hidden2'))
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax', name='predictions'))

opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

# Treinamento do modelo

In [ ]:
batch_size = 32
epochs = 30

history = model.fit(
    datagen.flow(trainX, trainY, batch_size=batch_size),
    validation_data=(testX, testY),
    epochs=epochs
)

In [ ]:
loss, acc = model.evaluate(testX, testY)
print(f"Test Accuracy: {acc:.4f}")

In [ ]:
model.save("model_cifar10.h5")

# Testando modelo no conjunto de teste

In [ ]:
plt.imshow(testX[10], cmap='gray');

In [ ]:
pred = np.argmax(model.predict(np.expand_dims(testX[10], axis=0)))

pred

In [ ]:
cifar10_labels = {
    0: 'airplane',
    1: 'automobile',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck'
}

'Predição: {pred}'.format(pred=cifar10_labels[pred])